In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
"""
pulls data from Data directories, turns the selected csv into a Pandas dataframe, and removes columns with NaNs (if present)

parameters:
    year -- year of data being pulled
    division -- division whose data is being pulled
    stat -- determines which of the three sheets in the directory will be pulled (0 = champion, 1 = player, 2 = team)
"""

#TODO: account for inconsistencies in division names

def get_dat(year, division, stat):
    # Check if folder exists
    if Path("./Data/" + str(year) + "/" + division).exists() == False or len(os.listdir("./Data/" + str(year) + "/" + division)) == 0:
        return pd.DataFrame()
    
    year_form = str(year) + "/"
    div_form = division + "/"
    
    if(division == "MSI" and (int(year) >= 2017 and int(year) < 2021)):
        msi_div = "main-event/"
        res_df = pd.read_csv("./Data/" + year_form + div_form + msi_div + os.listdir("./Data/" + year_form + div_form + msi_div)[stat])
    else:
        res_df = pd.read_csv("./Data/" + year_form + div_form + os.listdir("./Data/" + year_form + div_form)[stat])
        
    return res_df.dropna(axis = 1, how = "all")

In [3]:
divs = ["CBLOL", "LCK", "LCL", "LCO", "LCS", "LEC", "LJL", "LLA", "LPL", "MSI", "PCS", "TCL", "VCS"]

res_df = pd.DataFrame()

for year in os.listdir("./Data"):
    for div in divs:
        if get_dat(year, div, 2).empty:
            continue
        else:
            df = get_dat(year, div, 2)
            df = df.assign(
                win_rate = round((df["W"] / (df["W"] + df["L"])) * 100, 2)
            ).assign(
                year = year
            ).assign(
                division = div
            )
            
            res_df = pd.concat([df, res_df])
            
res_df = res_df.reset_index(drop = True)
res_df = res_df.sort_values(by = ["year", "Team"]).fillna(0)
res_df.head()

,Team,GP,W,L,AGT,K,D,KD,CKPM,GPR,...,FBN%,BN%,LNE%,JNG%,WPM,CWPM,WCPM,win_rate,year,division
371,7th heaven,13,7,6,42.7,239,235,1.02,0.85,0.00,...,0,40%,0,0,0.00,0.00,0.00,53.85,2015,LJL
341,ATLAS eSports Team,14,6,8,35.8,214,269,0.80,0.96,-0.89,...,43%,53%,49.5%,44.6%,2.21,0.32,0.69,42.86,2015,TCL
367,Bencheados,6,1,5,39.3,80,121,0.66,0.85,0.00,...,0,0,0,0,0.00,0.00,0.00,16.67,2015,LLA
342,Beşiktaş Esports,14,12,2,33.9,291,175,1.66,0.98,1.47,...,64%,88%,52.0%,50.6%,2.37,0.34,0.82,85.71,2015,TCL
354,Beşiktaş Esports,5,0,5,27.6,27,110,0.25,0.99,-4.84,...,0%,0%,45.7%,38.7%,2.82,0.27,0.60,0.00,2015,MSI


In [4]:
# Store and drop columns we don't want to be converted
dontConvert = ['Team', 'division', 'EGR', 'MLR', 'year']
columns = res_df.get(dontConvert)
data = res_df.drop(dontConvert, axis = 1)

In [5]:
# Convert percents to decimal form
for col in data:
    if data.dtypes[col] == 'object':
        data[col] = data[col].str.rstrip('%').astype('float') / 100.0
data = data.fillna(0)
data.head()

,GP,W,L,AGT,K,D,KD,CKPM,GPR,GSPD,...,DRG%,ELD%,FBN%,BN%,LNE%,JNG%,WPM,CWPM,WCPM,win_rate
371,13,7,6,42.7,239,235,1.02,0.85,0.00,0.000,...,0.56,0.0,0.00,0.40,0.000,0.000,0.00,0.00,0.00,53.85
341,14,6,8,35.8,214,269,0.80,0.96,-0.89,-0.062,...,0.26,0.0,0.43,0.53,0.495,0.446,2.21,0.32,0.69,42.86
367,6,1,5,39.3,80,121,0.66,0.85,0.00,0.000,...,0.36,0.0,0.00,0.00,0.000,0.000,0.00,0.00,0.00,16.67
342,14,12,2,33.9,291,175,1.66,0.98,1.47,0.153,...,0.62,0.0,0.64,0.88,0.520,0.506,2.37,0.34,0.82,85.71
354,5,0,5,27.6,27,110,0.25,0.99,-4.84,-0.326,...,0.25,0.0,0.00,0.00,0.457,0.387,2.82,0.27,0.60,0.00


In [6]:
data = data.join(columns).reset_index()
data.head()

,index,GP,W,L,AGT,K,D,KD,CKPM,GPR,...,JNG%,WPM,CWPM,WCPM,win_rate,Team,division,EGR,MLR,year
0,371,13,7,6,42.7,239,235,1.02,0.85,0.00,...,0.000,0.00,0.00,0.00,53.85,7th heaven,LJL,0.0,0.0,2015
1,341,14,6,8,35.8,214,269,0.80,0.96,-0.89,...,0.446,2.21,0.32,0.69,42.86,ATLAS eSports Team,TCL,0.0,0.0,2015
2,367,6,1,5,39.3,80,121,0.66,0.85,0.00,...,0.000,0.00,0.00,0.00,16.67,Bencheados,LLA,0.0,0.0,2015
3,342,14,12,2,33.9,291,175,1.66,0.98,1.47,...,0.506,2.37,0.34,0.82,85.71,Beşiktaş Esports,TCL,0.0,0.0,2015
4,354,5,0,5,27.6,27,110,0.25,0.99,-4.84,...,0.387,2.82,0.27,0.60,0.00,Beşiktaş Esports,MSI,0.0,0.0,2015


In [24]:
msi_place = list(range(len(data))) # creates blank list to put values to use for msi_placed data column

for i in range(1, len(data) - 1): # loops through rows of data
    if data.Team[i] == data.Team[i + 1] or data.Team[i - 1] == data.Team[i]: 
        # checking to see if a duplicate of a team name appears, meaning they made it to MSI for that split
        if data.division[i] == "MSI":
            msi_place[i] = -1 # assigns a team's MSI performace the indicator value of -1
            continue
        else:
            msi_place[i] = 1 # the team made it to MSI
    else:
        msi_place[i] = 0 # the team didn't

if data.division[398] == "MSI": # loop above does not check last row of data because of out of bounds error, but does same thing
    msi_place[398] = -1
else:
    msi_place[398] = 1

data["msi_placed"] = msi_place # with the completed list of indicator values, paste it into a new column in the dataframe
      
data

,index,GP,W,L,AGT,K,D,KD,CKPM,GPR,...,WPM,CWPM,WCPM,win_rate,Team,division,EGR,MLR,year,msi_placed
0,371,13,7,6,42.7,239,235,1.02,0.85,0.00,...,0.00,0.00,0.00,53.85,7th heaven,LJL,0.0,0.0,2015,0
1,341,14,6,8,35.8,214,269,0.80,0.96,-0.89,...,2.21,0.32,0.69,42.86,ATLAS eSports Team,TCL,0.0,0.0,2015,0
2,367,6,1,5,39.3,80,121,0.66,0.85,0.00,...,0.00,0.00,0.00,16.67,Bencheados,LLA,0.0,0.0,2015,0
3,342,14,12,2,33.9,291,175,1.66,0.98,1.47,...,2.37,0.34,0.82,85.71,Beşiktaş Esports,TCL,0.0,0.0,2015,1
4,354,5,0,5,27.6,27,110,0.25,0.99,-4.84,...,2.82,0.27,0.60,0.00,Beşiktaş Esports,MSI,0.0,0.0,2015,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,89,8,4,4,30.7,97,84,1.15,0.74,-0.01,...,3.25,1.37,1.41,50.00,Vorax Liberty,CBLOL,48.0,2.0,2021,0
395,28,6,2,4,32.4,68,114,0.60,0.94,-0.91,...,3.53,1.55,1.44,33.33,paiN Gaming,MSI,38.6,-5.3,2021,-1
396,87,14,9,5,31.7,198,191,1.04,0.88,0.57,...,3.68,1.49,1.46,64.29,paiN Gaming,CBLOL,63.1,1.2,2021,1
397,10,18,14,4,32.3,305,191,1.60,0.85,1.42,...,2.78,1.17,1.39,77.78,İstanbul Wildcats,TCL,67.8,9.9,2021,1
